<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질-답, 모두의노코드 질-답+댓 각각을 요약해 저장하는 코드(로컬)

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import urllib.request
!pip install datasets

In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load Model and Tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

In [ ]:
from datasets import Dataset, load_dataset
import torch
from google.colab import drive
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#카카오톡 질답세션 불러오고 Dataset으로 받기
file_path = '/content/drive/My Drive/judge_answer_result_pongjin_roberta_with_kornli_range10_qstandard0.63_astandard0.6.csv'

df1 = pd.read_csv(file_path, names = ["question", "answer"], header = None)

dataset = load_dataset("csv", data_files=file_path, column_names=["question", "answer"])

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#모두의노코드 질-답+댓 세션 불러오고 Dataset으로 받기
file_path = '/content/drive/My Drive/community_preprocess_result.csv'

df2 = pd.read_csv(file_path, names = ["question", "answer"], header = None)

dataset = load_dataset("csv", data_files = file_path, column_names = ["question", "answer"])

In [ ]:
# #두 데이터프레임 합치고 Dataset으로 변환하기
# df1.append(df2)
dataset = Dataset.from_pandas(df1)

In [ ]:
# Ensure the classifier is using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

In [ ]:
# 데이터셋 전처리 (예제)
texts = ["This is a long text that needs to be summarized.", "Here's another text for summarization."]
tokenized_inputs = tokenizer(texts, padding="longest", truncation=True, return_tensors="pt", max_length=1024)

print(tokenized_inputs)

In [ ]:
# 데이터셋 질문 전처리 함수, Dataset 열을 인자로 받는다.
def preprocess_question(examples):
    return tokenizer(examples['question'], return_tensors = "pt", truncation=True, padding="max_length", max_length=1026)


In [ ]:
#데이터를 gpu로 이동시킬 때 DataLoader에서 사용되는 collate_fn 함수
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids'], dtype = torch.long) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask'], dtype = torch.long) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

In [ ]:
#모델로 question summary 리스트 생성
question_summary = []

# Encoding question
question_text_dict = {"question" : dataset["question"][1:]} #dataset["question"]의 첫 요소인 'question'삭제
question_text = Dataset.from_dict(question_text_dict)
tokenized_question_inputs = question_text.map(preprocess_question, num_proc = 4, batched = True, batch_size = 128)

#데이터를 gpu로 이동시킬 때 DataLoader에서 사용되는 collate_fn 함수
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids'], dtype = torch.long) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask'], dtype = torch.long) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

loader = DataLoader(tokenized_question_inputs, batch_size=16, collate_fn=collate_fn)

# Question summary ids 생성,배치 단위
for batch in loader:
  #배치 단위로 loader에서 gpu로 데이터 전송
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  question_summary_ids = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty = 1.0,
    max_length = 40,
    min_length = 5,
    num_beams = 6,
    repetition_penalty = 1.5,
    no_repeat_ngram_size = 3,
  )
  #생성한 Question summary ids를 다시 tokenizer가 있는 cpu로 이동
  question_summary_ids = question_summary_ids.cpu()

  #배치마다 answer_summary_ids를 생성하기 때문에 cpu로 옮겨진 answer_summary_ids 배치 1개를 answer_summary 리스트에 옮기기
  question_summary.append(question_summary_ids)

# Decoding question summary (list)
qsum = []

#배치 단위로 decode 처리
for question_summary_result in question_summary:
  qsum_batch = [tokenizer.decode(question_summary_result[i], skip_special_tokens = True) for i in tqdm(range(len(question_summary_result)), desc = "Decoding question summary")]
  qsum.append(qsum_batch)

#배치 풀기
qsum = sum(qsum, [])

print(qsum[:5])


In [ ]:
# 데이터셋 답변 전처리 함수, Dataset 열을 인자로 받는다. 질문 전처리 함수와 공통적으로 사용하지 않은 이유는 examples['answer']형태로 입력하지 않으면 오류가 발생한다.
def preprocess_answer(examples):
    return tokenizer(examples['answer'], return_tensors = "pt", truncation=True, padding="max_length", max_length=1026)


In [ ]:
#모델로 answer summary 리스트 생성
answer_summary = []

# Encoding answer
answer_text_dict = {"answer" : dataset["answer"][1:]} #dataset["answer"]의 첫 요소인 'answer'삭제
answer_text = Dataset.from_dict(answer_text_dict)
tokenized_answer_inputs = answer_text.map(preprocess_answer, num_proc = 4, batched = True, batch_size = 128)

loader = DataLoader(tokenized_answer_inputs, batch_size=16, collate_fn=collate_fn)

# Answer summary ids 생성,배치 단위, question때와 다른 하이퍼파라미터 필요(요약길이)
for batch in loader:
  #배치 단위로 loader에서 gpu로 데이터 전송
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  answer_summary_ids = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty = 1.0,
    max_length = 70,
    min_length = 5,
    num_beams = 6,
    repetition_penalty = 1.5,
    no_repeat_ngram_size = 3,
  )

  #생성한 Answer summary ids를 다시 tokenizer가 있는 cpu로 이동
  answer_summary_ids = answer_summary_ids.cpu()

  #배치마다 answer_summary_ids를 생성하기 때문에 cpu로 옮겨진 answer_summary_ids 배치 1개를 answer_summary 리스트에 옮기기
  answer_summary.append(answer_summary_ids)

# Decoding question summary (list)
asum = []

#배치 단위로 decode 처리
for answer_summary_result in answer_summary:
  asum_batch = [tokenizer.decode(answer_summary_result[i], skip_special_tokens = True) for i in tqdm(range(len(answer_summary_result)), desc = "Decoding answer summary")]
  asum.append(asum_batch)

#배치 풀기
asum = sum(asum, [])

print(asum[:5])


Map (num_proc=4):   0%|          | 0/1696 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(asum,  columns = ['answer_summary'])
print(df)

In [ ]:
#데이터셋에 새로운 열 추가
print(len(dataset))

dataset_dict = {'question' : dataset['question'][1:], 'answer' : dataset['answer'][1:], 'question_summary' : qsum, 'answer_summary' : asum}
dataset = Dataset.from_dict(dataset_dict)

print(dataset)
# dataset.add_column("qsum", qsum)
# dataset.add_column("asum", asum)

In [ ]:
#.csv 파일로 google drive에 저장
save_path = '/content/drive/My Drive/summary_result.csv'

dataset.to_csv(save_path)